In [1]:
import threading
print(threading.active_count())

5


In [2]:
print(threading.enumerate())

[<_MainThread(MainThread, started 548259434512)>, <Thread(Thread-2, started daemon 548190073328)>, <Heartbeat(Thread-3, started daemon 548181680624)>, <HistorySavingThread(IPythonHistorySavingThread, started 548156502512)>, <ParentPollerUnix(Thread-1, started daemon 547809653232)>]


In [3]:
from jetbot import Robot
from jetbot import Camera
from jetbot import bgr8_to_jpeg

from time import sleep
import ipywidgets.widgets as widgets
import traitlets

import numpy as np
import cv2

In [4]:
import traitlets
from traitlets.config.configurable import SingletonConfigurable
import atexit
import cv2
import threading
import numpy as np


class Camera2(SingletonConfigurable):
    
    value = traitlets.Any()
    
    # config
    width = traitlets.Integer(default_value=224).tag(config=True)
    height = traitlets.Integer(default_value=224).tag(config=True)
    fps = traitlets.Integer(default_value=21).tag(config=True)
    capture_width = traitlets.Integer(default_value=3280).tag(config=True)
    capture_height = traitlets.Integer(default_value=2464).tag(config=True)

    def __init__(self, *args, **kwargs):
        self.value = np.empty((self.height, self.width, 3), dtype=np.uint8)
        super(Camera2, self).__init__(*args, **kwargs)

        try:
            self.cap = cv2.VideoCapture(self._gst_str(), cv2.CAP_GSTREAMER)

            re, image = self.cap.read()

            if not re:
                raise RuntimeError('Could not read image from camera.')

            self.value = image
            self.start()
        except:
            self.stop()
            raise RuntimeError(
                'Could not initialize camera.  Please see error trace.')

        atexit.register(self.stop)

    def _capture_frames(self):
        while True:
            re, image = self.cap.read()
            if re:
                self.value = image
            else:
                break
                
    def _gst_str(self):
        return 'nvarguscamerasrc ! video/x-raw(memory:NVMM), width=%d, height=%d, format=(string)NV12, framerate=(fraction)%d/1 ! nvvidconv ! video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! videoconvert ! appsink' % (
                self.capture_width, self.capture_height, self.fps, self.width, self.height)
    
    def start(self):
        if not self.cap.isOpened():
            self.cap.open(self._gst_str(), cv2.CAP_GSTREAMER)
        if not hasattr(self, 'thread') or not self.thread.isAlive():
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if hasattr(self, 'cap'):
            self.cap.release()
        if hasattr(self, 'thread'):
            self.thread.join()
            
    def restart(self):
        self.stop()
        self.start()

In [5]:
camera = Camera2.instance(width=640, height=480)

In [6]:
camW = 640
camH = 480
camera = Camera2.instance(width=camW, height=camH)

In [124]:
ROIWidth = camW
ROIHeight = 50
ROILeftUpX = 0
ROILeftUpY = 150
# ROILeftUpX = 150
# ROILeftUpY = 305
ROIRightBottomX = ROILeftUpX + ROIWidth
ROIRightBottomY = ROILeftUpY + ROIHeight


In [133]:
print("wait for 2 seconds...")
# sleep(1)
for i in range(1):
    img = camera.value
    hsvArray = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    markedImageArray = img
    cropped = img[ROILeftUpY:ROIRightBottomY, ROILeftUpX:ROIRightBottomX]
    croppedHSV = hsvArray[ROILeftUpY:ROIRightBottomY, ROILeftUpX:ROIRightBottomX]
    # mark ROI (BGR)
    borderWIDTH = 1
    # left border
    markedImageArray[ROILeftUpY:ROIRightBottomY, ROILeftUpX:ROILeftUpX+borderWIDTH, 0] = 0
    markedImageArray[ROILeftUpY:ROIRightBottomY, ROILeftUpX:ROILeftUpX+borderWIDTH, 1] = 0
    markedImageArray[ROILeftUpY:ROIRightBottomY, ROILeftUpX:ROILeftUpX+borderWIDTH, 2] = 255
    # top border
    markedImageArray[ROILeftUpY:ROILeftUpY+borderWIDTH, ROILeftUpX:ROIRightBottomX, 0] = 0
    markedImageArray[ROILeftUpY:ROILeftUpY+borderWIDTH, ROILeftUpX:ROIRightBottomX, 1] = 0
    markedImageArray[ROILeftUpY:ROILeftUpY+borderWIDTH, ROILeftUpX:ROIRightBottomX, 2] = 255
    # right border
    markedImageArray[ROILeftUpY:ROIRightBottomY, ROIRightBottomX-borderWIDTH:ROIRightBottomX, 0] = 0
    markedImageArray[ROILeftUpY:ROIRightBottomY, ROIRightBottomX-borderWIDTH:ROIRightBottomX, 1] = 0
    markedImageArray[ROILeftUpY:ROIRightBottomY, ROIRightBottomX-borderWIDTH:ROIRightBottomX, 2] = 255
    # bottom border
    markedImageArray[ROIRightBottomY-borderWIDTH:ROIRightBottomY, ROILeftUpX:ROIRightBottomX, 0] = 0
    markedImageArray[ROIRightBottomY-borderWIDTH:ROIRightBottomY, ROILeftUpX:ROIRightBottomX, 1] = 0
    markedImageArray[ROIRightBottomY-borderWIDTH:ROIRightBottomY, ROILeftUpX:ROIRightBottomX, 2] = 255
    
    # YELLOW - find column with the most yellow pixels
    # Yellow H value of HSV color space: 60 in range of [0,360], 30 in range of [0,180] in openCV
    # Yellow Range: H:[25,35] S:[100,255] V:[100,255]
    maskYellow = ((20 <= croppedHSV[:,:,0]) & (croppedHSV[:,:,0] <= 40)) \
                    & (croppedHSV[:,:,1] >= 50) & (croppedHSV[:,:,2] >= 50)
    yellows = maskYellow.sum(axis=0)
    laneYellow = yellows.argmax()
    
    # WHITE - find column with the most white pixels
    # White Range: H:[0,255] S:[0,255] V:[255,255]
    # if the saturation is low and the value is high
    maskWhite = (croppedHSV[:,:,0] <= 0) \
               & (croppedHSV[:,:,1] <= 0) & (croppedHSV[:,:,2] >= 255)
    whites = maskWhite.sum(axis=0)
    laneWhite = whites.argmax()
    laneWhite = 0
    # GREEN - find column with the most green pixels
    # Green H value of HSV color space: 120 in range of [0,360], 60 in range of [0,180] in openCV
    # Green Range: H:[55,65] S:[100,255] V:[100,255]
    maskGreen = (40 <= croppedHSV[:,:,0]) & (croppedHSV[:,:,0] <= 80) \
                    & (croppedHSV[:,:,1] >= 100) & (croppedHSV[:,:,2] >= 100)
    greens = maskGreen.sum(axis=0)
    laneGreen = greens.argmax()
    
    # RED - find column with the most red pixels
    # Red H value of HSV color space: 0~10 and  in range of [0,360], 60 in range of [0,180] in openCV
    # Red Range: H:[55,65] S:[100,255] V:[100,255]
    maskRed = (((0 <= croppedHSV[:,:,0]) & (croppedHSV[:,:,0] <= 20)) | \
               ((160 <= croppedHSV[:,:,0]) & (croppedHSV[:,:,0] <= 180))) & \
                (croppedHSV[:,:,1] >= 50) & (croppedHSV[:,:,2] >= 50)
    reds = maskRed.sum(axis=0)
    laneRed = reds.argmax()

    # BLUE
    # Blue Range: H:[110,130] S:[50,255] V:[50,255]
    maskBlue = ((100 <= croppedHSV[:,:,0]) & (croppedHSV[:,:,0] <= 140)) & \
                (croppedHSV[:,:,1] >= 50) & (croppedHSV[:,:,2] >= 50)
    blues = maskBlue.sum(axis=0)
    laneBlue = blues.argmax()
    
    # mark Yellow
    yellowColumn = ROILeftUpX + laneYellow
    markedImageArray[:,yellowColumn-1:yellowColumn, 0] = 0
    markedImageArray[:,yellowColumn-1:yellowColumn, 1] = 255
    markedImageArray[:,yellowColumn-1:yellowColumn, 2] = 255

    # mark White
    whiteColumn = ROILeftUpX + laneWhite
    markedImageArray[:,whiteColumn-1:whiteColumn, 0] = 255
    markedImageArray[:,whiteColumn-1:whiteColumn, 1] = 255
    markedImageArray[:,whiteColumn-1:whiteColumn, 2] = 255

    # mark Green
    greenColumn = ROILeftUpX + laneGreen
    markedImageArray[:,greenColumn-1:greenColumn, 0] = 0
    markedImageArray[:,greenColumn-1:greenColumn, 1] = 255
    markedImageArray[:,greenColumn-1:greenColumn, 2] = 0
    
    # mark Red
    redColumn = ROILeftUpX + laneRed
    markedImageArray[:,redColumn-1:redColumn, 0] = 0
    markedImageArray[:,redColumn-1:redColumn, 1] = 0
    markedImageArray[:,redColumn-1:redColumn, 2] = 255
    
    # mark Blue
    blueColumn = ROILeftUpX + laneBlue
    markedImageArray[:,blueColumn-1:blueColumn, 0] = 255
    markedImageArray[:,blueColumn-1:blueColumn, 1] = 0
    markedImageArray[:,blueColumn-1:blueColumn, 2] = 0
    
    prefix = "./img/img"
    suffix = ".jpg"
#     cv2.imwrite(prefix+str(i)+suffix, camera.value)
#     cv2.imwrite(prefix+str(i)+"crop"+suffix, cropped)
    cv2.imwrite(prefix+str(i)+"mark"+suffix, markedImageArray)
    cv2.imwrite(prefix+str(i)+"hsv"+suffix, croppedHSV)
    print(prefix+str(i)+suffix+" saved")
    sleep(0.5)

wait for 2 seconds...
./img/img0.jpg saved


asf

In [148]:
a = croppedHSV[:,:,0].argmax()

In [149]:
a

869

In [153]:
croppedHSV.shape

(50, 200, 3)

In [48]:
np.where(croppedHSV == np.max(croppedHSV))

(array([ 0,  0,  0, ..., 82, 92, 97]),
 array([ 2,  3,  4, ..., 36, 24, 35]),
 array([1, 1, 1, ..., 1, 1, 1]))

In [49]:
croppedHSV[97][35][1]

255

In [ ]:
img = cv2.imread("img/img0mark.jpg")
cv2.imshow("hello", img)

In [6]:
import traitlets
from traitlets.config.configurable import SingletonConfigurable
import atexit
import cv2
import threading
import numpy as np


class Camera2(SingletonConfigurable):
    
    value = traitlets.Any()
    
    # config
    width = traitlets.Integer(default_value=224).tag(config=True)
    height = traitlets.Integer(default_value=224).tag(config=True)
    fps = traitlets.Integer(default_value=21).tag(config=True)
    capture_width = traitlets.Integer(default_value=3280).tag(config=True)
    capture_height = traitlets.Integer(default_value=2464).tag(config=True)

    def __init__(self, *args, **kwargs):
        self.value = np.empty((self.height, self.width, 3), dtype=np.uint8)
        super(Camera, self).__init__(*args, **kwargs)

        try:
            self.cap = cv2.VideoCapture(self._gst_str(), cv2.CAP_GSTREAMER)

            re, image = self.cap.read()

            if not re:
                raise RuntimeError('Could not read image from camera.')

            self.value = image
            self.start()
        except:
            self.stop()
            raise RuntimeError(
                'Could not initialize camera.  Please see error trace.')

        atexit.register(self.stop)

    def _capture_frames(self):
        while True:
            re, image = self.cap.read()
            if re:
                self.value = image
            else:
                break
                
    def _gst_str(self):
        return 'nvarguscamerasrc ! video/x-raw(memory:NVMM), width=%d, height=%d, format=(string)NV12, framerate=(fraction)%d/1 ! nvvidconv ! video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! videoconvert ! appsink' % (
                self.capture_width, self.capture_height, self.fps, self.width, self.height)
    
    def start(self):
        if not self.cap.isOpened():
            self.cap.open(self._gst_str(), cv2.CAP_GSTREAMER)
        if not hasattr(self, 'thread') or not self.thread.isAlive():
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if hasattr(self, 'cap'):
            self.cap.release()
        if hasattr(self, 'thread'):
            self.thread.join()
            
    def restart(self):
        self.stop()
        self.start()

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=250)
image = widgets.Image(format='jpeg', width=300, height=250)
cropped = camera.value[0:30, 0:300]
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((cropped, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

TypeError: Each object must be HasTraits, not <class 'numpy.ndarray'>

In [9]:
camera.start()

In [18]:
for i in range(10):
    r, f = camera.cap.read()
    g = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('./img/image{}.png'.format(i), g)
#     sleep(0.5)

In [ ]:
# cv2.imshow('frame', g)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [3]:
# while(camera.cap.isOpened()):  # check !
#     # capture frame-by-frame
#     ret, frame = camera.cap.read()

#     if ret: # check ! (some webcam's need a "warmup")
#         # our operation on frame come here
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#         # Display the resulting frame
#         cv2.imshow('frame', gray)
#         cv2.waitKey(0)
#         cv2.destroyWindow('frame')
#     else:
#         print('image frame not detected')
        
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# # When everything is done release the capture
# camera.cap.release()
# cv2.destroyAllWindows()

image detected
image detected
image detected
image detected
image detected
image detected
image detected
image detected
image detected
image detected
image detected
image detected
image detected
image detected
image detected


error: /home/nvidia/build_opencv/opencv/modules/highgui/src/window_gtk.cpp:1198: error: (-215) found && "Can't destroy non-registered window" in function cvDestroyWindow


In [5]:
camera.stop()